# Data Generator

> Fill in a module description hered

In [ ]:
# | default_exp data_generator

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
from typing import List, Callable

import torch

from toolformer.api import APIBase

In [ ]:
#| export
class DataGenerator:
    def __init__(self, configs: dict, apis: List[APIBase]):
        self.start_token = configs["augmentation_data"]["start_token"]
        self.end_token = configs["augmentation_data"]["end_token"]
        self.sampling_threshold = configs["augmentation_data"]["sampling_threshold"]
        self.filtering_threshold = configs["augmentation_data"]["filtering_threshold"]
        self.apis = apis
        self.api_positions: List[int] = []
    
    def generate(
        self,
        prompt: str,
        model: Callable,
        tokenizer: Callable
    ) -> List[str]:
        prompt_ids = tokenizer(prompt, return_tensors="pt")["input_ids"][0]
        generated_ids = prompt_ids
        
        # sampling
        with torch.no_grad():    
            while True:
                logits = model(
                    input_ids=generated_ids,
                ).logits

                last_token = logits[-1, :]
                probs = torch.softmax(last_token, dim=-1)
                
                # find the top k tokens for api call
                top_k_tokens = torch.topk(probs, k=5, dim=-1).indices
                api_ids = torch.where(top_k_tokens == self.start_token)[0]
                
                if api_ids.size(0) > 0:
                    api_positions = torch.cat((api_positions, api_ids), dim=0)
                
                # sampling a token
                next_token = torch.multinomial(probs, num_samples=1)
                generated_ids = torch.cat((generated_ids, next_token), dim=0)
                
                print(f"api_ids: {api_ids}")
                print(f"top_k: {top_k_tokens}")
                
                if tokenizer.eos_token_id in top_k_tokens:
                    break
        
        # filtering
        
        # return